In [1]:
import rasterio as rio
import geopandas as gpd
import pandas as pd
import os, sys, glob
from osgeo import gdal
from rasterstats import zonal_stats
from rasterio.warp import reproject, Resampling
from shapely.geometry import box
from rasterio.mask import mask
import numpy as np
from rasterio import features
from shapely.geometry import shape, Polygon
from shapely.wkt import loads
from tqdm import tqdm
import geojson, json
import osmnx as ox
from affine import Affine
from matplotlib import pyplot as plt
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point

In [2]:
sys.path.append("/home/wb514197/Repos/INFRA_SAP")
import infrasap.rasterMisc as rMisc
import infrasap.aggregator as agg

In [3]:
data_dir = '/home/wb514197/data/ethiopia'
dig_dir = os.path.join(data_dir, "digitize_africa")

In [4]:
woreda = os.path.join(data_dir, 'admin', 'gadm', 'ETH_adm3.shp')
flood_map = os.path.join(data_dir, "flood", "flood_buff.tif")

In [5]:
target = gpd.read_file(woreda)
flood = rio.open(flood_map)

In [7]:
flood_crs = flood.crs.to_string()

In [9]:
target = target.to_crs(flood_crs)

In [10]:
flood_dict = {
    0:'non-flooded',
    1:"flooded"
}

In [11]:
flood_data = flood.read(1)

In [12]:
meta = flood.meta

In [13]:
zs_area = pd.DataFrame(zonal_stats(target, flood_data, affine=meta['transform'], categorical=True, category_map=flood_dict))

/home/wb514197/.conda/envs/geo/lib/python3.7/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [14]:
target.columns

Index(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'NAME_1', 'ID_2', 'NAME_2', 'ID_3',
       'NAME_3', 'TYPE_3', 'ENGTYPE_3', 'NL_NAME_3', 'VARNAME_3', 'geometry'],
      dtype='object')

In [15]:
area = target[['NAME_3','ID_3']].join(zs_area)

In [17]:
area.loc[:,"cells"] = area['non-flooded']+area['flooded']
area.loc[:,'Area_Cells'] = area['cells']*0.0004
area.loc[:,'Area_Flooded'] = area['flooded']*0.0004
area.loc[:,'Area_Flooded_pct'] = area['Area_Flooded']/area['Area_Cells']

In [19]:
out_folder = os.path.join(data_dir, "output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [20]:
area.to_csv(os.path.join(out_folder, "area_flooded.csv"))

In [21]:
def standardizeInputRasters(inR1, inR2, data_type="N"):
    ''' Standardize inR1 to inR2: changes crs, extent, and resolution.
    Inputs:
    inR1, inR2 [rasterio raster object]
    inR1_outFile [string] - output file for creating inR1 standardized to inR2
    [optional] data_type [string ['C','N']]
    
    Returns:
    nothing
    '''
    if inR1.crs != inR2.crs:
        bounds = gpd.GeoDataFrame(pd.DataFrame([[1, box(*inR2.bounds)]], columns=["ID","geometry"]), geometry='geometry', crs=inR2.crs)
        bounds = bounds.to_crs(inR1.crs)
        b2 = bounds.total_bounds
        boxJSON = [{'type': 'Polygon', 'coordinates': [[[b2[0], b2[1]],[b2[0], b2[3]],[b2[2], b2[3]],[b2[2], b2[1]],[b2[0], b2[1]]]]}]
    else:
        b2 = inR2.bounds
        boxJSON = [{'type': 'Polygon', 'coordinates': [[[b2.left, b2.bottom],[b2.left, b2.top],[b2.right, b2.top],[b2.right, b2.bottom],[b2.left, b2.bottom]]]}]
    
    #Clip R1 to R2
    #Get JSON of bounding box
    out_img, out_transform = mask(inR1, boxJSON, crop=True)
    out_meta = inR1.meta.copy()
    #Re-scale resolution of R1 to R2
    newArr = np.empty(shape=(1, inR2.shape[0], inR2.shape[1]))
    
    if data_type == "N":
        resampling_type = Resampling.cubic
    elif data_type == "C":
        resampling_type = Resampling.nearest
    
    reproject(out_img, newArr, src_transform=out_transform, dst_transform=inR2.transform, src_crs=inR1.crs, dst_crs=inR2.crs, resampling=resampling_type)
    out_meta.update({"driver": "GTiff",
                     "height": newArr.shape[1],
                     "width": newArr.shape[2],
                     "transform": inR2.transform,
                     "crs": inR2.crs})
    
    return(newArr, out_meta)

In [22]:
flood_mask = (flood_data==1)

In [23]:
allFeatures = []
idx = 0
for cShape, value in features.shapes(flood_data, mask=flood_mask, transform=flood.meta['transform']):
    allFeatures.append([idx, shape(geojson.loads(json.dumps(cShape)))])
    idx+=1

In [25]:
xx = pd.DataFrame(allFeatures, columns=['ID', 'geometry'])
xxGeom = gpd.GeoDataFrame(xx, geometry='geometry', crs = flood_crs)

In [26]:
xxGeom.to_file(os.path.join(out_folder, "flood_bool.shp"), driver="ESRI Shapefile")

In [27]:
cci_map = os.path.join(data_dir, "cci", "land_cover_ETH.tif")

In [28]:
cci = rio.open(cci_map)

In [29]:
flood_re, flood_re_meta = standardizeInputRasters(flood, cci, "C")

In [30]:
cci_data = cci.read(1)

In [31]:
flood_cci = flood_re[0,:,:]

In [32]:
cci_data.shape == flood_cci.shape

True

In [33]:
cci_flooded = cci_data*flood_cci

In [34]:
cci.crs

CRS.from_epsg(4326)

In [36]:
target = target.to_crs(cci.crs)

In [37]:
cci_dict = {
    1:'trees',
    2:"shrubs",
    3:"grasslands",
    4:"cropland",
    5:"vegetation aquatic",
    6:"sparse vegetation",
    7:"bare areas",
    8:"built up",
    10:"water"
}

In [38]:
zs = pd.DataFrame(zonal_stats(target, cci_flooded, affine=cci.meta['transform'], categorical=True, nodata=0, category_map=cci_dict))

In [40]:
zs.fillna(0, inplace=True)

In [42]:
zs_tot = pd.DataFrame(zonal_stats(target, cci_data, affine=cci.meta['transform'], categorical=True, nodata=0, category_map=cci_dict))

In [43]:
zs_tot.fillna(0, inplace=True)

In [44]:
zs_shares = zs.div(zs_tot)

In [45]:
cci_pcts = zs.div(zs.sum(axis=1), axis=0)

In [46]:
zs['all'] = zs.sum(axis=1)

In [48]:
cci_area = zs*0.0004

In [49]:
cci_res = cci_area.join(cci_pcts, rsuffix="_pct")

In [50]:
cci_res2 = cci_res.join(zs_shares, rsuffix="_shares")

In [51]:
target.columns

Index(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'NAME_1', 'ID_2', 'NAME_2', 'ID_3',
       'NAME_3', 'TYPE_3', 'ENGTYPE_3', 'NL_NAME_3', 'VARNAME_3', 'geometry'],
      dtype='object')

In [52]:
cci_res2 = target[['NAME_3', 'ID_3']].join(cci_res2)

In [53]:
cci_res2.fillna(0, inplace=True)

In [54]:
cci_res2.to_csv(os.path.join(out_folder, "CCI Flood Stats.csv"))